In [1]:

import os
import numpy as np



def load_video_features(base_dir1,base_dir2,base_dir3):
    data = []
    labels = []
    class_names = sorted(os.listdir(base_dir1))

    for class_idx,class_name in enumerate(class_names):
        class_dir1 = os.path.join(base_dir1, class_name)
        class_dir2 = os.path.join(base_dir2, class_name)
        class_dir3 = os.path.join(base_dir3, class_name)
        video_files = sorted(os.listdir(class_dir1))
        print (f'processing class={class_idx}:{class_name}')

        for video_file in video_files:
            features = np.load(os.path.join(class_dir1, video_file))
            features1 = features.reshape(features.shape[0], -1)
            
            features = np.load(os.path.join(class_dir2, video_file))
            features2 = features.reshape(features.shape[0], -1) 

            features = np.load(os.path.join(class_dir3, video_file))
            features3 = features.reshape(features.shape[0], -1) 
            # video_features = np.load(video_path, allow_pickle=True)
            # features = video_features.item()['resnet']
            

            # data.append(features1)
            # data.append(features2)
            # data.append(features3)
            # labels.append(class_idx)
            # labels.append(class_idx)
            # labels.append(class_idx)

            new_feature=np.concatenate((features1,features2,features3),axis=1)
            data.append(new_feature)
            labels.append(class_idx)

    return data, labels,class_names

# Example usage
base_dir1 ='F:\Downloads\RIP_dataset\VGG16_Training_Features\\left_side_mirror_view'
base_dir2 ='F:\Downloads\RIP_dataset\VGG16_Training_Features\\frontal_view'
base_dir3 ='F:\Downloads\RIP_dataset\VGG16_Training_Features\\right_side_mirror_view'
video_features_train, video_labels_train, class_names = load_video_features(base_dir1,base_dir2,base_dir3)
video_labels_train=np.array(video_labels_train)
print(video_features_train[0].shape)


processing class=0:Left Lane Change
processing class=1:Left Turn
processing class=2:Right Lane Change
processing class=3:Right Turn
processing class=4:Slow-Stop
processing class=5:Straight
(300, 1536)


In [2]:
def pad_videos(video_features):
    max_frames = max(video.shape[0] for video in video_features)
    max_frames=300
    padded_videos = []

    for video in video_features:
        num_frames = video.shape[0]
        if num_frames < max_frames:
            padding = np.zeros((max_frames - num_frames, video.shape[1]))
            #padding = np.zeros((max_frames - num_frames))
            padded_video = np.concatenate((video, padding), axis=0)
        elif num_frames > max_frames:
            selected_indices =sorted(np.random.choice(num_frames, max_frames, replace=False))
            padded_video = video[selected_indices]
        else:
            padded_video = video
        padded_videos.append(padded_video)

    return np.array(padded_videos), max_frames



In [3]:
X_vgg_train, max_frames_train = pad_videos(video_features_train)
print("trin padded feature: ",X_vgg_train.shape)

trin padded feature:  (500, 300, 1536)


In [6]:
X_train_flat = X_vgg_train.reshape(X_vgg_train.shape[0], -1)
print("X_train:",X_train_flat.shape)

X_train: (500, 460800)


In [7]:
y_train=video_labels_train


In [8]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_flat, y_train)
print("Original shape:", X_train_flat.shape)
print("Resampled shape:", X_train_resampled.shape)
print("Original labels distribution:", np.bincount(y_train))
print("Resampled labels distribution:", np.bincount(y_train_resampled))

Original shape: (500, 460800)
Resampled shape: (960, 460800)
Original labels distribution: [ 29 160  39 154  44  74]
Resampled labels distribution: [160 160 160 160 160 160]


In [10]:
y_train=y_train_resampled


In [11]:
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.calibration import CalibratedClassifierCV

class_weights = {0: 6.0, 1: 1.5, 2: 2.5, 3: 0.8, 4: 3.0, 5: 1}

ova_clf = OneVsRestClassifier(svm.SVC(kernel='rbf', probability=True, class_weight=class_weights,random_state=42))
ova_clf.fit(X_train_resampled, y_train)


OneVsRestClassifier(estimator=SVC(class_weight={0: 6.0, 1: 1.5, 2: 2.5, 3: 0.8,
                                                4: 3.0, 5: 1},
                                  probability=True, random_state=42))

In [ ]:
import joblib
# Save the model
joblib.dump(ova_clf, 'ova_clf_mixmodel69_50.pkl')

In [1]:
#if model is already trained ...load the model from  here
import joblib
model=joblib.load('ova_clf_mixmodel70_50.pkl')

<h2>validation data

In [12]:
import os
import numpy as np

def load_video_features(base_dir1,base_dir2,base_dir3):
    data = []
    labels = []
    class_names = sorted(os.listdir(base_dir1))

    for class_idx,class_name in enumerate(class_names):
        class_dir1 = os.path.join(base_dir1, class_name)
        class_dir2 = os.path.join(base_dir2, class_name)
        class_dir3 = os.path.join(base_dir3, class_name)
        video_files = sorted(os.listdir(class_dir1))
        print (f'processing class={class_idx}:{class_name}')

        for video_file in video_files:
            features = np.load(os.path.join(class_dir1, video_file))
            features1 = features.reshape(features.shape[0], -1)
            
            features = np.load(os.path.join(class_dir2, video_file))
            features2 = features.reshape(features.shape[0], -1) 

            features = np.load(os.path.join(class_dir3, video_file))
            features3 = features.reshape(features.shape[0], -1) 
            # video_features = np.load(video_path, allow_pickle=True)
            # features = video_features.item()['resnet']
            # data.append(features1)
            # data.append(features2)
            # data.append(features3)
            # labels.append(class_idx)
            # labels.append(class_idx)
            # labels.append(class_idx)
            new_feature=np.concatenate((features1,features2,features3),axis=1)
            data.append(new_feature)
            labels.append(class_idx)

    return data, labels,class_names


# Example usage
base_dir1 ='F:\Downloads\RIP_dataset\VGG16_val_features\\left_side_mirror_view'
base_dir2 ='F:\Downloads\RIP_dataset\VGG16_val_features\\frontal_view'
base_dir3 ='F:\Downloads\RIP_dataset\VGG16_val_features\\right_side_mirror_view'
video_features_val, video_labels_val, class_names = load_video_features(base_dir1,base_dir2,base_dir3)
video_labels_val=np.array(video_labels_val)
print(video_features_val[0].shape)


processing class=0:Left Lane Change
processing class=1:Left Turn
processing class=2:Right Lane Change
processing class=3:Right Turn
processing class=4:Slow-Stop
processing class=5:Straight
(210, 1536)


In [13]:
X_vgg_val,_ = pad_videos(video_features_val)
print("val padded feature: ",X_vgg_val.shape)

val padded feature:  (200, 300, 1536)


In [14]:
del(video_features_val)
X_val_flat = X_vgg_val.reshape(X_vgg_val.shape[0], -1)
print("X_val:",X_val_flat.shape)

X_val: (200, 460800)


In [16]:
y_val=video_labels_val

In [17]:
print(y_val)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]


In [19]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

proba = ova_clf.predict_proba(X_val_flat)  
pred = np.argmax(proba, axis=1)
print(pred)


accuracy = accuracy_score(y_val,pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

f1 = f1_score(y_val, pred, average='micro')
print(f'F1 Score: {f1:.2f}')

[0 1 1 2 2 0 5 0 2 4 2 2 2 0 1 2 2 1 2 3 1 1 1 1 3 1 0 1 3 0 1 1 1 3 3 1 1
 3 1 3 1 1 3 1 1 1 1 1 1 1 1 1 1 1 2 1 3 4 2 2 2 5 2 4 4 1 0 2 4 0 1 2 2 3
 3 3 3 3 3 1 3 3 3 1 3 3 3 5 3 3 1 2 3 3 5 3 3 3 3 3 3 3 3 1 3 3 3 3 1 3 1
 3 3 3 3 3 3 3 3 3 2 3 3 4 4 4 4 4 1 4 2 5 1 2 4 2 4 3 2 3 3 3 4 4 3 4 3 4
 1 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 4 5 5 5 5 5 5 5 5
 5 5 5 5 5 4 5 5 5 5 5 5 5 5 5]
Accuracy: 69.50%
F1 Score: 0.69


In [29]:
import os
import numpy as np



def load_video_features(base_dir1,base_dir2,base_dir3):
    file= []
    class_names = sorted(os.listdir(base_dir1))

    for class_idx,class_name in enumerate(class_names):
        class_dir1 = os.path.join(base_dir1, class_name)
        class_dir2 = os.path.join(base_dir2, class_name)
        class_dir3 = os.path.join(base_dir3, class_name)
        video_files = sorted(os.listdir(class_dir1))
        print (f'processing class={class_idx}:{class_name}')

        for video_file in video_files:
            file.append(os.path.splitext(video_file)[0])

    return file


# Example usage
base_dir1 ='F:\Downloads\RIP_dataset\VGG16_val_features\\left_side_mirror_view'
base_dir2 ='F:\Downloads\RIP_dataset\VGG16_val_features\\frontal_view'
base_dir3 ='F:\Downloads\RIP_dataset\VGG16_val_features\\right_side_mirror_view'
files = load_video_features(base_dir1,base_dir2,base_dir3)


processing class=0:Left Lane Change
processing class=1:Left Turn
processing class=2:Right Lane Change
processing class=3:Right Turn
processing class=4:Slow-Stop
processing class=5:Straight


In [30]:
import numpy as np
import pandas as pd


num_classes = 6

# Convert class indices to one-hot encoding
one_hot_pred = np.zeros((len(pred), num_classes))
one_hot_pred[np.arange(len(pred)), pred] = 1

# Column names
columns = ['frontal_view_video_name', 'Left Lane Change', 'Left Turn', 'Right Lane Change', 'Right Turn', 'Slow-Stop', 'Straight']

# Create a DataFrame
df = pd.DataFrame(one_hot_pred, columns=columns[1:])
df.insert(0, 'frontal_view_video_name', files)

# Save to CSV
csv_file_path = 'task2_val_predictions.csv'
df.to_csv(csv_file_path, index=False)

print(f"CSV file saved to {csv_file_path}")


CSV file saved to task2_val_predictions.csv


<h2>test prediction


In [22]:
import os
import numpy as np



def load_video_features(base_dir1,base_dir2,base_dir3):
    data = []
    labels = []
    files=[]
  
    video_files = sorted(os.listdir(base_dir1))
    for video_file in video_files:
        features = np.load(os.path.join(base_dir1, video_file))
        features1 = features.reshape(features.shape[0], -1)
        
        features = np.load(os.path.join(base_dir2, video_file))
        features2 = features.reshape(features.shape[0], -1) 

        features = np.load(os.path.join(base_dir3, video_file))
        features3 = features.reshape(features.shape[0], -1) 
        # video_features = np.load(video_path, allow_pickle=True)
        # features = video_features.item()['resnet']
        # data.append(features1)
        # data.append(features2)
        # data.append(features3)
        # labels.append(class_idx)
        # labels.append(class_idx)
        # labels.append(class_idx)
        new_feature=np.concatenate((features1,features2,features3),axis=1)
        files.append(os.path.splitext(video_file)[0])
        data.append(new_feature)

    return data,files


# Example usage
base_dir1 ='F:\Downloads\RIP_dataset\VGG16_test_features\left_view'
base_dir2 ='F:\Downloads\RIP_dataset\VGG16_test_features\\frontal_view\VGG16_features'
base_dir3 ='F:\Downloads\RIP_dataset\VGG16_test_features\\right_view'
video_features_test,files = load_video_features(base_dir1,base_dir2,base_dir3)
print(video_features_test[0].shape)
print(list(files[i] for i in range(5)))

(82, 1536)
['004ed8d4-5be9-4963-bdb3-8aace317cd74', '00d5a64d-7e78-4662-9f00-c2a7c69cace8', '01bd1017-7748-4e0b-bf57-d9012563695a', '02d9a67f-cd75-4880-9e41-08b9369e5ded(1)', '02d9a67f-cd75-4880-9e41-08b9369e5ded']


In [23]:
X_vgg_test,_ = pad_videos(video_features_test)
print("val padded feature: ",X_vgg_test.shape)

val padded feature:  (299, 300, 1536)


In [24]:
del(video_features_test)
X_test_flat = X_vgg_test.reshape(X_vgg_test.shape[0], -1)
print("X_val:",X_test_flat.shape)

X_val: (299, 460800)


In [26]:
from sklearn.metrics import accuracy_score

proba = ova_clf.predict_proba(X_test_flat) 
test_pred = np.argmax(proba, axis=1)
print(test_pred)


[1 3 5 2 2 1 1 3 0 1 4 4 2 4 2 3 1 3 5 5 5 3 5 3 3 1 3 4 1 1 3 4 5 4 1 5 3
 1 2 5 1 5 5 2 2 5 2 5 0 1 1 2 2 2 5 3 3 1 2 2 5 2 5 1 1 5 5 1 1 1 5 1 1 1
 4 3 1 1 1 5 0 5 5 1 4 3 0 4 2 5 4 3 1 4 5 4 1 3 5 5 1 5 1 1 5 1 5 2 5 0 4
 0 1 4 0 1 1 3 3 3 5 5 2 1 0 5 5 2 1 4 2 5 5 1 1 1 2 2 1 1 1 4 1 2 1 4 1 1
 4 3 1 5 5 3 1 1 3 4 5 5 3 1 1 1 2 3 5 2 4 4 2 5 3 1 4 5 5 4 2 1 1 4 1 3 3
 5 4 4 3 3 1 3 2 1 3 3 1 4 1 1 5 1 1 3 1 5 5 1 2 1 3 1 3 4 2 1 5 2 5 1 5 1
 1 5 5 3 4 2 5 5 1 5 1 2 5 4 5 1 5 1 2 1 4 5 1 4 2 1 5 1 1 5 1 3 3 4 2 2 5
 5 5 5 5 1 5 1 5 1 5 1 0 5 1 1 5 4 2 5 4 1 5 5 3 5 1 2 3 1 2 1 5 4 5 1 4 2
 1 1 2]


In [27]:
import numpy as np
import pandas as pd

# Assuming `files` is a list of filenames and `pred` is a list of predicted class indices

# Example predictions as class indices

# Number of classes
num_classes = 6

# Convert class indices to one-hot encoding
one_hot_pred = np.zeros((len(test_pred), num_classes))
one_hot_pred[np.arange(len(test_pred)), test_pred] = 1

# Column names
columns = ['frontal_view_video_name', 'Left Lane Change', 'Left Turn', 'Right Lane Change', 'Right Turn', 'Slow-Stop', 'Straight']

# Create a DataFrame
df = pd.DataFrame(one_hot_pred, columns=columns[1:])
df.insert(0, 'frontal_view_video_name', files)

# Save to CSV
csv_file_path = 'task2_test_predictions.csv'
df.to_csv(csv_file_path, index=False)

print(f"CSV file saved to {csv_file_path}")


CSV file saved to task2_test_predictions.csv
